# Email OSINT Finder

This notebook provides a multi-input wrapper for [holehe](https://github.com/megadose/holehe) and [ignorant](https://github.com/megadose/ignorant), both of which are OSINT-discovery utilities that check if an email or phone number is attached to an account on sites like twitter, instagram, imgur and more than 120 others (phone lookups supported for Amazon, Instagram, and Snapchat). 

Emails will not be notified of the check. Output is provided in either CSV or JSON format.

### IMPORTANT: If phone numbers are selected as source, all available permutations (country codes) will be attempted and results will indicate which country codes provided positive results!

TODO: Add output (other than DF)

In [3]:

from ipywidgets import widgets, Layout, HBox, Label

style = {'description_width': 'initial'}

source = widgets.RadioButtons(
    options=['Email', 'Phone Number'],
    value='Email',
    description='Source Type:',
    disabled=False,
    style=style
)
input_data = widgets.Textarea(
    value='',
    description='Input Data:',
    layout=Layout(width='60%', height='80px'),
    style=style
)
delimiter = widgets.RadioButtons(
    options=['[newline]', '[,]', '[, ]', '[|]'],
    value='[newline]',
    description='Delimiter:',
    disabled=False,
    style=style
)
output = widgets.RadioButtons(
    options=['CSV', 'JSON', 'Text (Raw)'],
    value='JSON',
    description='Output:',
    disabled=False,
    style=style
)

display(HBox([source, input_data, delimiter, output]))

In [14]:

import subprocess, re
from collections import OrderedDict
from tqdm.notebook import tqdm
import pandas as pd

MATCH = re.compile(r'\[\+\] ([a-z\-\.0-9]{2,36}\.[a-z]{2,4})')
CODE = {
    'FIFA': 'Dial',
    'TPE': '886',
    'AFG': '93',
    'ALB': '355',
    'ALG': '213',
    'ASA': '1-684',
    'AND': '376',
    'ANG': '244',
    'AIA': '1-264',
    'ROS': '672',
    'ATG': '1-268',
    'ARG': '54',
    'ARM': '374',
    'ARU': '297',
    'AUS': '61',
    'AUT': '43',
    'AZE': '994',
    'BAH': '1-242',
    'BHR': '973',
    'BAN': '880',
    'BRB': '1-246',
    'BLR': '375',
    'BEL': '32',
    'BLZ': '501',
    'BEN': '229',
    'BER': '1-441',
    'BHU': '975',
    'BOL': '591',
    'ANT': '599',
    'BIH': '387',
    'BOT': '267',
    'BRA': '55',
    'VGB': '1-284',
    'BRU': '673',
    'BUL': '359',
    'BFA': '226',
    'BDI': '257',
    'CPV': '238',
    'CAM': '855',
    'CMR': '237',
    'CAN': '1',
    'CAY': '1-345',
    'CTA': '236',
    'CHA': '235',
    'CHI': '56',
    'CHN': '86',
    'HKG': '852',
    'MAC': '853',
    'CXR': '61',
    'CCK': '61',
    'COL': '57',
    'COM': '269',
    'CGO': '242',
    'COK': '682',
    'CRC': '506',
    'CRO': '385',
    'CUB': '53',
    'CYP': '357',
    'CZE': '420',
    'CIV': '225',
    'PRK': '850',
    'COD': '243',
    'DEN': '45',
    'DJI': '253',
    'DMA': '1-767',
    'DOM': '1-809',
    'ECU': '593',
    'EGY': '20',
    'SLV': '503',
    'EQG': '240',
    'ERI': '291',
    'EST': '372',
    'SWZ': '268',
    'ETH': '251',
    'FLK': '500',
    'FRO': '298',
    'FIJ': '679',
    'FIN': '358',
    'FRA': '33',
    'GUF': '594',
    'TAH': '689',
    'GAB': '241',
    'GAM': '220',
    'GEO': '995',
    'GER': '49',
    'GHA': '233',
    'GBZ': '350',
    'GRE': '30',
    'GRL': '299',
    'GRN': '1-473',
    'GLP': '590',
    'GUM': '1-671',
    'GUA': '502',
    'GBG': '44',
    'GUI': '224',
    'GNB': '245',
    'GUY': '592',
    'HAI': '509',
    'VAT': '39-06',
    'HON': '504',
    'HUN': '36',
    'ISL': '354',
    'IND': '91',
    'IDN': '62',
    'IRN': '98',
    'IRQ': '964',
    'IRL': '353',
    'GBM': '44',
    'ISR': '972',
    'ITA': '39',
    'JAM': '1-876',
    'JPN': '81',
    'GBJ': '44',
    'JOR': '962',
    'KAZ': '7',
    'KEN': '254',
    'KIR': '686',
    'KUW': '965',
    'KGZ': '996',
    'LAO': '856',
    'LVA': '371',
    'LIB': '961',
    'LES': '266',
    'LBR': '231',
    'LBY': '218',
    'LIE': '423',
    'LTU': '370',
    'LUX': '352',
    'MAD': '261',
    'MWI': '265',
    'MAS': '60',
    'MDV': '960',
    'MLI': '223',
    'MLT': '356',
    'MHL': '692',
    'MTQ': '596',
    'MTN': '222',
    'MRI': '230',
    'MYT': '262',
    'MEX': '52',
    'FSM': '691',
    'MON': '377',
    'MNG': '976',
    'MNE': '382',
    'MSR': '1-664',
    'MAR': '212',
    'MOZ': '258',
    'MYA': '95',
    'NAM': '264',
    'NRU': '674',
    'NEP': '977',
    'NED': '31',
    'NCL': '687',
    'NZL': '64',
    'NCA': '505',
    'NIG': '227',
    'NGA': '234',
    'NIU': '683',
    'NFK': '672',
    'NMI': '1-670',
    'NOR': '47',
    'OMA': '968',
    'PAK': '92',
    'PLW': '680',
    'PAN': '507',
    'PNG': '675',
    'PAR': '595',
    'PER': '51',
    'PHI': '63',
    'PCN': '870',
    'POL': '48',
    'POR': '351',
    'PUR': '1',
    'QAT': '974',
    'KOR': '82',
    'MDA': '373',
    'ROU': '40',
    'RUS': '7',
    'RWA': '250',
    'REU': '262',
    'SHN': '290',
    'SKN': '1-869',
    'LCA': '1-758',
    'SPM': '508',
    'VIN': '1-784',
    'SAM': '685',
    'SMR': '378',
    'STP': '239',
    'KSA': '966',
    'SEN': '221',
    'SRB': '381',
    'SEY': '248',
    'SLE': '232',
    'SIN': '65',
    'SVK': '421',
    'SVN': '386',
    'SOL': '677',
    'SOM': '252',
    'RSA': '27',
    'ESP': '34',
    'SRI': '94',
    'PLE': '970',
    'SUD': '249',
    'SUR': '597',
    'SWE': '46',
    'SUI': '41',
    'SYR': '963',
    'TJK': '992',
    'THA': '66',
    'MKD': '389',
    'TLS': '670',
    'TOG': '228',
    'TKL': '690',
    'TGA': '676',
    'TRI': '1-868',
    'TUN': '216',
    'TUR': '90',
    'TKM': '993',
    'TCA': '1-649',
    'TUV': '688',
    'UGA': '256',
    'UKR': '380',
    'UAE': '971',
    'ENG': 'NIR',
    'TAN': '255',
    'VIR': '1-340',
    'USA': '1',
    'URU': '598',
    'UZB': '998',
    'VAN': '678',
    'VEN': '58',
    'VIE': '84',
    'WLF': '681',
    'SAH': '212',
    'YEM': '967',
    'ZAM': '260',
    'ZIM': '263',
    'ALD': '358'
 }

def format_phone(entry):
    if entry.startswith('+'):
        entry = entry.lstrip('+')
    results = []
    for key, value in CODE.items():
        if entry.startswith(value):
            results.append({
                'country': key,
                'prefix': value,
                'number': entry.lstrip(value)
            })
    return results

if delimiter.value == '[newline]':
    in_data = [s for s in input_data.value.split('\n')]
else:
    in_data = [s for s in input_data.value.split(delimiter.value.lstrip('[').rstrip(']'))]

out_data = []

for entry in tqdm(in_data):
    if entry:
        # print('Processing [%s of %s]: %s...' %(), end='\r')
        if source.value == 'Email':
            response = subprocess.run([f'holehe {entry} --only-used --no-color'], shell=True, capture_output=True)
            matches = re.findall(MATCH, response.stdout.decode('utf-8'))
            out_data.append({
                'entry': entry,
                'results': matches,
                'num_results': len(matches)
            })
        else:
            permutations = format_phone(entry)
            for permutation in permutations:
                prefix = permutation['prefix']
                number = permutation['number']
                response = subprocess.run([f'ignorant {prefix} {number} --only-used --no-color'], shell=True, capture_output=True)
                matches = re.findall(MATCH, response.stdout.decode('utf-8'))
                out_data.append({
                    'entry': entry,
                    'country': permutation['country'],
                    'results': matches,
                    'num_results': len(matches)
                })
        # print(f'Processing: {entry}... done! Matches: {len(matches)}')

sorted_data = sorted(out_data, key=lambda x: x['num_results'], reverse=True) 

printable_data = []
for d in sorted_data:
    d['results'] = ', '.join(d['results'])
    printable_data.append(d)

pd.set_option('display.max_colwidth', 0)
df = pd.DataFrame(printable_data)
df


  0%|          | 0/1 [00:00<?, ?it/s]

""
